In [1]:
# This tests PynPoint

# created 2018 Sept. 20 by E.S.

In [1]:
import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.Hdf5Reading import Hdf5ReadingModule
from PynPoint.IOmodules.FitsWriting import FitsWritingModule
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ContrastCurveModule, \
                                       FluxAndPosition

In [17]:
working_place = "./pynpoint_experimentation/working_place/"
input_place = "./pynpoint_experimentation/input_place/"
output_place = "./pynpoint_experimentation/output_place/"

#url = urllib.URLopener()
#url.retrieve("https://people.phys.ethz.ch/~stolkert/BetaPic_NACO_Mp.hdf5",
#             working_place+"BetaPic_NACO_Mp.hdf5")

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

# PynPoint_config.ini should be initiated now

Initiating PynPoint... [DONE]


In [18]:
# read HDF5 file of science frames

read_science = Hdf5ReadingModule(name_in="read_science",
                         input_filename="BetaPic_NACO_Mp.hdf5",
                         input_dir=None,
                         tag_dictionary={"stack":"stack"})

pipeline.add_module(read_science)

In [19]:
# read FITS file of unsaturated PSF for fake planet injection

read_model_psf = FitsReadingModule(name_in="read_model_psf",
                                 input_dir=input_place+"/fake_unsat",
                                 image_tag="model_psf",
                                 check=True)

pipeline.add_module(read_model_psf)

In [5]:
# inject fake planets

'''
inject = FluxAndPosition.FakePlanetModule(position=(0.46,0),
                          magnitude=0,
                          psf_scaling=1,
                          name_in="inject",
                          image_in_tag="stack",
                          psf_in_tag="model_psf",
                          image_out_tag="fake_planet_output")

pipeline.add_module(inject)
'''

'\ninject = FluxAndPosition.FakePlanetModule(position=(0.46,0),\n                          magnitude=0,\n                          psf_scaling=1,\n                          name_in="inject",\n                          image_in_tag="stack",\n                          psf_in_tag="model_psf",\n                          image_out_tag="fake_planet_output")\n\npipeline.add_module(inject)\n'

In [6]:
# write out science frames with fake planet

'''
write_inject = FitsWritingModule(file_name="junk_stack_fake_planet.fits",
                              name_in="write_inject",
                              output_dir=output_place,
                              data_tag="fake_planet_output")

pipeline.add_module(write_inject)
'''

'\nwrite_inject = FitsWritingModule(file_name="junk_stack_fake_planet.fits",\n                              name_in="write_inject",\n                              output_dir=output_place,\n                              data_tag="fake_planet_output")\n\npipeline.add_module(write_inject)\n'

In [20]:
# prepare psf


prep_fake_planet = PSFpreparationModule(name_in="prep_fake_planet",
                            image_in_tag="stack",
                            image_out_tag="prep",
                            image_mask_out_tag=None,
                            mask_out_tag=None,
                            norm=False,
                            resize=None,
                            cent_size=0.15,
                            edge_size=1.1)

pipeline.add_module(prep_fake_planet)

In [21]:
pipeline.run()

# after running the pipeline, hdf5 attributes like PIXSCALE for groups
# other than /config/ should be available

Validating Pypeline... [DONE]
Running Hdf5ReadingModule... [DONE]
Running FitsReadingModule... [DONE]
Running PSFpreparationModule... [DONE]


In [22]:
pixscale_config = pipeline.get_attribute("config", "PIXSCALE")
print(pixscale_config)
pixscale_prep = pipeline.get_attribute("prep", "PIXSCALE")
print(pixscale_prep)

0.0107
0.027


In [7]:
print(pixscale_config)

0.027


In [8]:
# write out prepped psf

'''
# checks HDF5 file
write_prep = FitsWritingModule(file_name="junk_prep.fits",
                              name_in="write_prep",
                              output_dir=output_place,
                              data_tag="prep")

#pipeline.add_module(write_prep)
'''

'\n# checks HDF5 file\nwrite_prep = FitsWritingModule(file_name="junk_prep.fits",\n                              name_in="write_prep",\n                              output_dir=output_place,\n                              data_tag="prep")\n\n#pipeline.add_module(write_prep)\n'

In [7]:
# do PCA PSF subtraction

'''
pca = PcaPsfSubtractionModule(pca_numbers=(5, ),
                              name_in="pca",
                              images_in_tag="fake_planet_output",
                              reference_in_tag="fake_planet_output",
                              res_mean_tag="mean_residuals",
                              res_median_tag="median_residuals",
                              res_arr_out_tag="all_resids",
                              res_rot_mean_clip_tag="resid_rot",
                              verbose=True)

pipeline.add_module(pca)

# note:
# images_in_tag: science images
# reference_in_tag: reference images, which COULD be the science images
'''

In [7]:
## THIS IS TEST ONLY ## do PCA PSF subtraction

'''
pca = PcaPsfSubtractionModule(pca_numbers=(5, ),
                              name_in="pca",
                              images_in_tag="read_science",
                              reference_in_tag="read_science",
                              res_mean_tag="mean_residuals",
                              res_median_tag="median_residuals",
                              res_arr_out_tag="all_resids",
                              res_rot_mean_clip_tag="resid_rot",
                              verbose=True)

pipeline.add_module(pca)
'''
# note:
# images_in_tag: science images
# reference_in_tag: reference images, which COULD be the science images

In [8]:
# write out outputs from PCA PSF subtraction

'''
# checks HDF5 file
read_test1 = FitsWritingModule(file_name="junk_mean_residuals.fits",
                              name_in="read_test1",
                              output_dir=output_place,
                              data_tag="mean_residuals")

pipeline.add_module(read_test1)
'''

In [9]:
# write out outputs from PCA PSF subtraction

'''
# checks HDF5 file
read_test2 = FitsWritingModule(file_name="junk_median_residuals.fits",
                              name_in="read_test2",
                              output_dir=output_place,
                              data_tag="median_residuals")

pipeline.add_module(read_test2)
'''

In [10]:
# write out outputs from PCA PSF subtraction

'''
# checks HDF5 file
read_test4 = FitsWritingModule(file_name="junk_resid_rot.fits",
                              name_in="read_test4",
                              output_dir=output_place,
                              data_tag="resid_rot")

pipeline.add_module(read_test4)
'''

In [6]:
# make a contrast curve

'''
cent_size: mask radius
'''

contrast_curve = ContrastCurveModule(name_in="contrast_curve",
                            image_in_tag="prep",
                            psf_in_tag="model_psf",
                            contrast_out_tag="contrast_landscape",
                            pca_out_tag="pca_resids",
                            pca_number=20,
                            psf_scaling=1,
                            separation=(0.1, 1.0, 0.1), 
                            angle=(0.0, 360.0, 60.0), 
                            magnitude=(7.5, 1.0),
                            cent_size=None)

pipeline.add_module(contrast_curve)

In [9]:
pipeline.run()

Validating Pypeline... [DONE]


AssertionError: Error: Input file does not exist. Input requested: BetaPic_NACO_Mp.hdf5

In [11]:
contrast_curve_results = pipeline.get_data("contrast_landscape")
#residuals = pipeline.get_data("residuals")
#pixscale = pipeline.get_attribute("stack", "PIXSCALE")

#size = pixscale*np.size(residuals, 1)/2.

In [12]:
# contrast curve

# [0]: separation
# [1]: azimuthally averaged contrast limits
# [2]: the azimuthal variance of the contrast limits
# [3]: threshold of the false positive fraction associated with sigma

contrast_curve_results

array([[1.00000000e-01,            nan,            nan, 3.74521694e-03],
       [2.00000000e-01,            nan,            nan, 2.68666801e-04],
       [3.00000000e-01, 5.07657121e+00, 1.80844893e-01, 6.54338490e-05],
       [4.00000000e-01, 4.73267286e+00, 8.75829719e-02, 2.33216658e-05],
       [5.00000000e-01, 4.57947726e+00, 1.54244613e-02, 1.26831579e-05],
       [6.00000000e-01, 6.79627759e+00, 7.58776386e-02, 8.02141316e-06],
       [7.00000000e-01, 7.15955594e+00, 8.30756762e-02, 5.60807982e-06],
       [8.00000000e-01, 7.48905131e+00, 4.07430502e-02, 4.03083259e-06],
       [9.00000000e-01, 7.51861472e+00, 1.13346032e-02, 3.20495930e-06]])

In [20]:


f, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(contrast_curve_results[:,0],contrast_curve_results[:,1])
ax1.set_title('Azim. averaged contrast limit')
ax1.set_xlabel('Radius (asec)')
ax2.set_ylabel('del_mag')
ax2.plot(contrast_curve_results[:,0],contrast_curve_results[:,3])
ax2.set_title('Threshold of FPF')
ax2.set_xlabel('Radius (asec)')
f.savefig('test.png')

In [19]:
plt.imshow(residuals[0, ], origin='lower', extent=[size, -size, -size, size])
plt.title("beta Pic b - NACO M' - mean residuals")
plt.xlabel('R.A. offset [arcsec]', fontsize=12)
plt.ylabel('Dec. offset [arcsec]', fontsize=12)
plt.colorbar()
#plt.show()
plt.savefig(output_place+"residuals.png")

NameError: name 'residuals' is not defined

In [13]:
pipeline

In [19]:
np.size(residuals)

NameError: name 'residuals' is not defined

In [10]:
pipeline.add_module(writefits)

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/Core/Pypeline.py:257: UserWarning: Processing module names need to be unique. Overwriting module 'fits_write'.
  % module.name)


In [11]:
writefits.run()

Running FitsWritingModule... [DONE]
